In [77]:
import pandas as pd

dataset = pd.read_csv("datarecom_test.csv")
dataset.head()

,Kategori,Nama Bahan,Air (gram),Energi (kal),Protein (gram),Lemak (gram),Karbohidrat (gram),Serat (gram)
0,Buah segar,"Alpukat, segar",84.3,85,0.9,6.5,7.7,NaN
1,Buah segar,"Anggur hutan, segar",92.3,30,0.5,0.2,6.8,1.2
2,Buah segar,"Apel malang, segar",85.9,57,0.5,0.4,12.8,0.8
3,Buah segar,"Apel, segar",84.1,58,0.3,0.4,14.9,2.6
4,Buah segar,"Arbai, segar",89.9,37,0.8,0.5,8.3,NaN


In [78]:
import random

In [79]:
df = dataset.loc[dataset["Kategori"] == "Olahan daging"].reset_index(drop=True)
df.fillna(0.0, inplace=True)
df["Harga (Rp.)"] = [random.randint(10000, 100000) for _ in range(len(df))]
df.head()

,Kategori,Nama Bahan,Air (gram),Energi (kal),Protein (gram),Lemak (gram),Karbohidrat (gram),Serat (gram),Harga (Rp.)
0,Olahan daging,"Ayam, ampela, goreng",4.8,270,32.3,11.2,9.9,0.0,13444
1,Olahan daging,"Ayam, usus, goreng",4.6,473,45.2,26.3,13.9,0.0,28238
2,Olahan daging,"Bebek, daging, goreng",48.8,300,24.0,20.7,4.5,0.0,77718
3,Olahan daging,"Buaya, daging, dendeng, mentah",12.3,365,49.6,4.8,30.8,0.0,74987
4,Olahan daging,Ham,42.0,384,16.9,35.0,0.3,0.0,72474


### Binary Integer Programming

In [80]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

In [81]:
# Define data
foods   = df["Nama Bahan"].tolist() 
water   = df["Air (gram)"].tolist()
energy  = df["Energi (kal)"].tolist()
protein = df["Protein (gram)"].tolist()
fat     = df["Lemak (gram)"].tolist()
carbs   = df["Karbohidrat (gram)"].tolist()
fiber   = df["Serat (gram)"].tolist()
cost    = df["Harga (Rp.)"].tolist()

In [82]:
df.describe()

,Air (gram),Energi (kal),Protein (gram),Lemak (gram),Karbohidrat (gram),Serat (gram),Harga (Rp.)
count,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000
mean,56.930682,225.409091,19.884091,12.445455,7.912500,0.280682,52921.806818
std,24.047794,134.214328,14.408625,11.197207,10.306117,1.017644,26328.259764
min,3.600000,34.000000,2.300000,0.400000,0.000000,0.000000,13120.000000
25%,44.350000,115.250000,7.950000,4.725000,1.375000,0.000000,28263.500000
50%,57.700000,207.500000,16.000000,9.300000,3.950000,0.000000,56890.500000
75%,76.025000,295.000000,32.025000,16.025000,11.425000,0.000000,72858.250000
max,91.600000,588.000000,57.300000,49.500000,59.300000,7.500000,99757.000000


In [83]:
# define constraints
N_MENU = 1
budget= 25000
max_water = 50
max_cal = 1600
max_protein = 70
max_fat = 70
max_carbs = 100
max_fiber = 0.5

# define problem
problem = LpProblem("Mean_Plan_Optimization", LpMaximize)

# binary variables
x = LpVariable.dicts("Food", foods, cat="Binary")

# objective function
problem += lpSum((water[i] + energy[i] + protein[i] + fat[i] + carbs[i] + fiber[i]) * x[foods[i]] for i in range(len(foods))), "Total_Nutrients"

# constraints
problem += lpSum(x[foods[i]] for i in range(len(foods))) == N_MENU, "Exact_Selection"

problem += lpSum(cost[i]    * x[foods[i]] for i in range(len(foods))) <= budget, "Budget_Constraint"
problem += lpSum(water[i]   * x[foods[i]] for i in range(len(foods))) >= max_water, "Water_Constraint"
problem += lpSum(energy[i]  * x[foods[i]] for i in range(len(foods))) >= max_cal, "Calories_Constraint"
problem += lpSum(protein[i] * x[foods[i]] for i in range(len(foods))) >= max_protein, "Protein_Constraint"
problem += lpSum(fat[i]     * x[foods[i]] for i in range(len(foods))) >= max_fat, "Fat_Constraint"
problem += lpSum(carbs[i]   * x[foods[i]] for i in range(len(foods))) >= max_carbs, "Carbs_Constraint"
problem += lpSum(fiber[i]   * x[foods[i]] for i in range(len(foods))) >= max_fiber, "Fiber_Constraint"

# solving
problem.solve()

# display results
print("Optimal mean plan:")
for food in foods:
    if x[food].value() == 1:
        print(f" - {food}")

total_budget = sum(cost[i] * x[foods[i]].value() for i in range(len(foods)))
total_water = sum(water[i] * x[foods[i]].value() for i in range(len(foods)))
total_calories = sum(energy[i]* x[foods[i]].value() for i in range(len(foods)))
total_protein = sum(protein[i] * x[foods[i]].value() for i in range(len(foods)))
total_fat = sum(fat[i] * x[foods[i]].value() for i in range(len(foods)))
total_carbs = sum(carbs[i] * x[foods[i]].value() for i in range(len(foods)))
total_fiber = sum(fiber[i] * x[foods[i]].value() for i in range(len(foods)))
print(f"Total Budget: Rp. {total_budget}")
print(f"Total Water: {total_water} g")
print(f"Total Calories: {total_calories} g")
print(f"Total Protein: {total_protein} g")
print(f"Total Fat: {total_fat} g")
print(f"Total Carbs: {total_carbs} g")
print(f"Total Fiber: {total_fiber} g")


Optimal mean plan:
 - Ayam, ampela, goreng
 - Ayam, usus, goreng
 - Itik alabio, daging, dendeng, mentah
 - Sapi, abon
 - Ayam goreng kalasan, paha
 - Ayam goreng mbok berek dada
 - Beef burger
 - Pelolah manuk, masakan
Total Budget: Rp. 24999.99897899991
Total Water: -7376.906721500001 g
Total Calories: -23059.659364000006 g
Total Protein: 69.99999689999913 g
Total Fat: -2716.9422239 g
Total Carbs: 100.00000150000005 g
Total Fiber: 7.9 g


In [72]:
df.loc[df["Nama Bahan"] == "Buaya, daging, dendeng, mentah"]

,Kategori,Nama Bahan,Air (gram),Energi (kal),Protein (gram),Lemak (gram),Karbohidrat (gram),Serat (gram),Harga (Rp.)
3,Olahan daging,"Buaya, daging, dendeng, mentah",12.3,365,49.6,4.8,30.8,0.0,10642


In [71]:
df.loc[df["Nama Bahan"] == "Itik alabio, daging, dendeng, mentah"]

,Kategori,Nama Bahan,Air (gram),Energi (kal),Protein (gram),Lemak (gram),Karbohidrat (gram),Serat (gram),Harga (Rp.)
5,Olahan daging,"Itik alabio, daging, dendeng, mentah",12.7,496,30.3,33.5,18.4,0.0,11381


In [ ]:
#### BASELINE

from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Contoh data menu dan nilai gizi
foods = ["Ayam", "Tahu", "Tempe", "Telur", "Daging"]
protein = [20, 3, 5, 6, 15]  # Protein dalam gram
carbs = [0, 2, 3, 1, 0]      # Karbohidrat dalam gram
cost = [10000, 5000, 7000, 6000, 15000]  # Harga dalam satuan

# Batasan nutrisi
max_protein = 30  # Misalnya, kita ingin total protein <= 30 gram
max_carbs = 10    # Misalnya, kita ingin total karbohidrat <= 10 gram
budget = 19000    # Misalnya, total biaya <= 25000

# Jumlah menu yang ingin dipilih
N = 3  # Pilih tepat 3 menu

# Define the problem
problem = LpProblem("Meal_Selection_Optimization", LpMaximize)

# Define binary variables untuk setiap menu
x = LpVariable.dicts("Food", foods, cat="Binary")

# Objective function: maximize total protein and carbs combined
problem += lpSum((protein[i] + carbs[i]) * x[foods[i]] for i in range(len(foods))), "Total_Nutrients"

# Constraints
problem += lpSum(x[foods[i]] for i in range(len(foods))) == N, "Exact_Selection"  # Harus memilih tepat N menu
problem += lpSum(protein[i] * x[foods[i]] for i in range(len(foods))) <= max_protein, "Max_Protein_Constraint"
problem += lpSum(carbs[i] * x[foods[i]] for i in range(len(foods))) <= max_carbs, "Max_Carbs_Constraint"
problem += lpSum(cost[i] * x[foods[i]] for i in range(len(foods))) <= budget, "Budget_Constraint"

# Solve the problem
problem.solve()

# Display results
print("Optimal menu combination:")
for food in foods:
    if x[food].value() == 1:
        print(f" - {food}")

# Display total nutrients selected
total_protein = sum(protein[i] * x[foods[i]].value() for i in range(len(foods)))
total_carbs = sum(carbs[i] * x[foods[i]].value() for i in range(len(foods)))
print(f"Total Protein: {total_protein} g")
print(f"Total Carbs: {total_carbs} g")
print(f"Total Cost: {sum(cost[i] * x[foods[i]].value() for i in range(len(foods)))}")


Optimal menu combination:
 - Ayam
Total Protein: 20.0 g
Total Carbs: 0.0 g
Total Cost: 10000.0


In [17]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Define data
foods = ["Chicken Breast", "Eggs", "Tofu", "Almonds", "Lentils"]
protein = [20, 6, 8, 6, 9]
carbs = [0, 1, 2, 5, 12]
cost = [3, 1, 1.5, 2, 1]
calories = [200, 70, 100, 150, 80]

# Define constraints
budget = 5
max_calories = 400
min_protein = 15

# Define weights for protein and carbs
w_protein = 1
w_carbs = 1

# Define the problem
problem = LpProblem("Meal_Plan_Optimization", LpMaximize)

# Define binary variables
x = LpVariable.dicts("Food", foods, cat="Binary")

# Objective function: maximize total protein and carb intake
problem += lpSum((w_protein * protein[i] + w_carbs * carbs[i]) * x[foods[i]] for i in range(len(foods))), "Total_Nutrients"

# Constraints
problem += lpSum(cost[i] * x[foods[i]] for i in range(len(foods))) <= budget, "Budget_Constraint"
problem += lpSum(calories[i] * x[foods[i]] for i in range(len(foods))) <= max_calories, "Calories_Constraint"
problem += lpSum(protein[i] * x[foods[i]] for i in range(len(foods))) >= min_protein, "Protein_Requirement"

# Solve the problem
problem.solve()

# Display results
print("Optimal meal plan:")
for food in foods:
    if x[food].value() == 1:
        print(f" - {food}")
        
print(f"Total Protein: {sum(protein[i] * x[foods[i]].value() for i in range(len(foods)))} g")
print(f"Total Carbs: {sum(carbs[i] * x[foods[i]].value() for i in range(len(foods)))} g")
print(f"Total Cost: {sum(cost[i] * x[foods[i]].value() for i in range(len(foods)))} $")
print(f"Total Calories: {sum(calories[i] * x[foods[i]].value() for i in range(len(foods)))} kcal")


Optimal meal plan:
 - Chicken Breast
 - Eggs
 - Lentils
Total Protein: 35.0 g
Total Carbs: 13.0 g
Total Cost: 5.0 $
Total Calories: 350.0 kcal
